In [374]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from itertools import repeat
import random
#from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.firefox.service import Service

In [375]:
chrome_options = Options()
firefox_driver_path = 'C:/webdrivers/geckodriver.exe'
chrome_driver_path = 'C:/webdrivers/chromedriver.exe'
service = Service(chrome_driver_path)
browser = webdriver.Chrome(options=chrome_options, service=service) # For Chrome
#browser = webdriver.Firefox(options=chrome_options, service = service)

#Empty lists to collect the data from the tables
list_of_rows_odd = [] 
list_of_rows_even = []

urls = [] # List of all the page urls for scraping that we need from 2015 - 2021

In [376]:
# Generates our list of urls to parse into BS Url. 
def parse_url():  
    year = [*range(2015, 2022)]
    year.insert(2, 2016) # Inserting a repeat year as Max drove for both Torro Rosso And Red Bull in 2016
    year_check = 0
    for y in year:
        if y == 2015 or 2016 and year_check < 2 :
            constructor = 'toro-rosso'
            year_check += 1
        else:   
            constructor = 'red-bull'
        urls.append(f'https://www.statsf1.com/en/{constructor}/grand-prix-{y}.aspx')
    return urls


In [377]:
# Loads the page and gets the data from the table by the get_table_data_by_row() func call
def load_page(urls):
    parse_url()
    for url in urls: 
        browser.get(url)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        get_table_data_by_row(1, 2, list_of_rows_odd, soup)
        get_table_data_by_row(2, 2, list_of_rows_even, soup)
        print(url)
    return    

In [378]:
# Gets the table data by alternating rows to tackle the row span discrepency
def get_table_data_by_row(starting_row, row_increment, list_to_append, soup_):
    row_num = starting_row
    while True:
        element = soup_.select(f"tr:nth-of-type({row_num})")
        if len(element) >= 1:
            for i in range(0, len(element)):
                tr = element[i]
                tmp_list = []
                td = tr.select('tr > td')
                for y in range(0, len(td)):
                    tmp_list.append(td[y].text.strip())
                list_to_append.append(tmp_list)
            row_num += row_increment
        else:
            break

# <p align=center>Scraping The Data </p>


In [379]:
load_page(urls)

https://www.statsf1.com/en/toro-rosso/grand-prix-2015.aspx
https://www.statsf1.com/en/toro-rosso/grand-prix-2016.aspx
https://www.statsf1.com/en/red-bull/grand-prix-2016.aspx
https://www.statsf1.com/en/red-bull/grand-prix-2017.aspx
https://www.statsf1.com/en/red-bull/grand-prix-2018.aspx
https://www.statsf1.com/en/red-bull/grand-prix-2019.aspx
https://www.statsf1.com/en/red-bull/grand-prix-2020.aspx
https://www.statsf1.com/en/red-bull/grand-prix-2021.aspx


# <p align=center> Pre Processing Data </p>
## <p>Noticing that the data from the website is displayed rather annoyingly, in the form of span rows, I have decided to alternate the rows with the view of pasting together both list into a DataFrame. To acheieve this, I have to pre process the data to have it in the correct sizing, format to import into the DataFrame.  The odd rows in particular, the way the data was displayed resulted in some entries being not of the same length, thus being out of sync with most of the data pulled from this subsection of rows.  The best way I came about to solving this issue, was to isolate the rows that are not needed, in particular the empty lists, and those that took the header of the table, and remove them.  Then insert a generic entry into the lists that remain, to align the data with the rest.  I will go through and clean this data once It is active in the DataFrame</p>

In [380]:
odd_df = pd.DataFrame(list_of_rows_odd, columns=["id", "track", "team", "driver_num", "driver", "model", "engine", "tyre", "grid_pos", "fin_pos", "notes"]) 
print(odd_df.shape)

odd_df = odd_df.iloc[odd_df.index[46:68]].shift(periods=2, axis="columns")


(172, 11)


In [381]:
# Sanity check for the amount of lists with empty strings. 
len(list_of_rows_odd)
count = 0
for ls in list_of_rows_odd: 
    if ls[0] == '':
        count += 1
print(count)

8


In [382]:
tmp_list_of_rows_odd = [row.copy() for row in list_of_rows_odd]
tmp_list_of_rows_odd  # Making a copy of the list to have a backup incase something goes wrong.

[['n',
  'Grand Prix',
  'Team',
  'N°',
  'Driver',
  'Model',
  'Engine',
  'Tyre',
  'Grid',
  'Race'],
 ['167',
  'Australia',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '11',
  'ab',
  'Engine'],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['168',
  'Malaysia',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '6',
  '7',
  ''],
 ['169',
  'China',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '13',
  '17',
  'Engine'],
 ['170',
  'Bahrain',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '15',
  'ab',
  'Electrics'],
 ['171',
  'Spain',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '6',
  '11',
  ''],
 ['172',
  'Monaco',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '9',
  'ab',
  'Collision'],
 ['173',
  '

In [391]:
# Removing lists of empty strings that we have in the list, and the lists of table headers from the webpage, and also insterting generic data into the indices to make it uniform
for ls in tmp_list_of_rows_odd:
    if ls[0] == '':
        tmp_list_of_rows_odd.remove(ls)
    if ls[0] == 'n':
        tmp_list_of_rows_odd.remove(ls)
    if len(ls) <= 10:
        ls.insert(0,'id')
        ls.insert(1,'track')
        print(ls)
    else:
        pass
print(len(tmp_list_of_rows_odd))

['id', 'track', 'Red Bull Racing', '26', 'KVYAT Daniil', 'RB12', 'TAG Heuer', 'Pirelli', '15', '7', '']
156


In [392]:
uniformed_rows_odd = [row.copy() for row in tmp_list_of_rows_odd]

In [393]:
# Adding the year to the list throughout the seasons results.
races_per_season = {
    '2015' : "19",
    '2016' : "21",
    '20116' : "21",
    '2017' : "20",
    '2018' : "21",
    '2019' : "21",
    '2020' : "17",
    '2021' : "16"
}

# second_right_list = [[second_right_list[i][0], k, *second_right_list[i][1:]] # Create a new list with the first element in list, then add the key, then unpack the rest from index 1 onwards. 
#             for k,v in races_per_season.items()
#             for i in range(int(v))]

i = 0
for k,v in races_per_season.items():
    total_races = 0
    while total_races < int(v):
        uniformed_rows_odd[i].insert(1, k)
        total_races += 1
        i += 1


In [394]:
for row in uniformed_rows_odd: 
    print(len(row))

12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12


# <p align=center>Creating the first DataFrame</p>
## Now that we have pre processed our odd data, by adding the requried indices to make it conform to the main list we can create our first DataFrame.

In [397]:
odd_df = pd.DataFrame(uniformed_rows_odd, columns=["id","year", "track", "team", "driver_num", "driver", "model", "engine", "tyre", "grid_pos", "fin_pos", "notes"]) 
odd_df.iloc[49:100]

,id,year,track,team,driver_num,driver,model,engine,tyre,grid_pos,fin_pos,notes
49,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,3,2,
50,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,4,5,
51,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,4,3,
52,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,2,11,
53,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,7,7,
54,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,4,6,
55,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,3,2,
56,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,3,2,
57,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,4,ab,Gearbox
58,id,20116,track,Red Bull Racing,33,VERSTAPPEN Max,RB12,TAG Heuer,Pirelli,3,4,5s penalty
